In [4]:
import pandas as pd
import json
import time
import re
from openai import OpenAI
from tqdm import tqdm
from pydantic import BaseModel, ValidationError
from typing import Optional, Literal
import os
from dotenv import load_dotenv

In [11]:
# Разрешенные значения
SentimentVal = Literal["positive", "negative", "neutral"]
AspectVal = Literal["positive", "negative", "neutral", "null"]


class AspectsSchema(BaseModel):
    delivery: Optional[AspectVal] = None
    price: Optional[AspectVal] = None
    quality: Optional[AspectVal] = None
    functionality: Optional[AspectVal] = None
    service: Optional[AspectVal] = None


class ReviewResponse(BaseModel):
    sentiment: SentimentVal
    aspects: AspectsSchema
    summary: str

In [5]:
load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")
if not API_KEY:
    raise ValueError("API Key не найден! Проверьте файл .env")
BASE_URL = "https://openrouter.ai/api/v1"
MODEL_NAME = "mistralai/devstral-2512:free"
TARGET_SIZE = 6000

In [13]:
SYSTEM_PROMPT = """You are an expert Data Labeler.
Task: Analyze the customer review and extract structured data.

Output Schema (JSON strictly):
{
  "sentiment": "positive" | "negative" | "neutral",
  "aspects": {
    "delivery": "positive" | "negative" | "neutral" | null,
    "price": "positive" | "negative" | "neutral" | null,
    "quality": "positive" | "negative" | "neutral" | null,
    "functionality": "positive" | "negative" | "neutral" | null,
    "service": "positive" | "negative" | "neutral" | null
  },
  "summary": "Short summary in Russian (max 1 sentence)"
}

Definitions & Rules for Aspect Mapping:

1. **delivery**: Everything related to shipping, packaging, speed of delivery, courier service, box condition, or item availability in stock.
2. **price**: Mentions of cost, value for money, discounts, "expensive", "cheap", "worth it".
3. **quality**: Physical condition, build quality, materials, defects, scratches, sound quality (for audio), screen brightness (for displays), taste/smell (for consumables).
4. **functionality**: How well the product performs its main task.
   - For electronics: battery life, wifi, connection, speed, software, bugs.
   - For appliances (e.g., iron): steam power, heating speed, modes.
   - For others: ease of use, installation, does it work as described?
5. **service**: Interactions with the seller, support team, refunds, warranty issues, rude/polite staff.

If an aspect is not mentioned, set it to null.
Use ONLY the allowed enum values ("positive", "negative"). If mixed/neutral, use "neutral"."""

In [14]:
def prepare_balanced_dataset(csv_path, target_size=1000):
    """
    Загрузка CSV, очистка дублей и создание выборки 50/50 (позитив/негатив)
    """
    print("--- Подготовка данных ---")
    df = pd.read_csv(csv_path)

    # Удаление дубликатов текста
    df = df.drop_duplicates(subset=['text'])

    # Разделение на классы
    negatives = df[df['rating'].isin([1, 2, 3])]
    positives = df[df['rating'].isin([4, 5])]

    print(f"Всего доступно: Негатив={len(negatives)}, Позитив={len(positives)}")

    # Определение, сколько можно взять (не больше чем количество самого редкого класса)
    n_samples_per_class = min(len(negatives), target_size // 2)

    if n_samples_per_class == 0:
        print("ОШИБКА: Не найдено негативных отзывов!")
        return pd.DataFrame()

    print(f"Берем по {n_samples_per_class} отзывов каждого типа...")

    sample_neg = negatives.sample(n=n_samples_per_class, random_state=42)
    sample_pos = positives.sample(n=n_samples_per_class, random_state=42)

    # Объединение и перемешивание
    balanced_df = pd.concat([sample_neg, sample_pos]).sample(frac=1, random_state=42)
    print(f"Итоговый датасет: {len(balanced_df)} строк.")
    return balanced_df

In [15]:
def clean_json_string(text):
    if not text: return None
    text = re.sub(r"^```json\s*", "", text)
    text = re.sub(r"^```\s*", "", text)
    text = re.sub(r"\s*```$", "", text)
    return text.strip()

In [16]:
def generate_label_safe(text, client):
    # Экспоненциальная задержка при ошибках
    delay = 2
    for attempt in range(5):
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"Review: {text}"}
                ],
                temperature=0.1,
                response_format={"type": "json_object"},
                max_tokens=512
            )

            raw = response.choices[0].message.content
            cleaned_json_str = clean_json_string(raw)

            # Проверка на пустоту
            if not cleaned_json_str:
                print(f"⚠️ Attempt {attempt}: Model returned empty string or None.")
                continue

            # Парсинг JSON
            data_dict = json.loads(cleaned_json_str)

            # Валидация через Pydantic
            validated = ReviewResponse(**data_dict)

            return validated.model_dump(mode='json')

        except json.JSONDecodeError as je:
            print(f"JSON Error (Attempt {attempt}): {je}")
            continue

        except ValidationError as ve:
            print(f"Validation Error (Attempt {attempt}): {ve}")
            continue

        except Exception as e:
            if "429" in str(e):  # Rate Limit
                print(f"⚠️ Rate Limit. Sleep {delay}s...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"Error: {e}")
                time.sleep(1)

    return None

In [17]:
def main():
    # Подготовка данных
    df = prepare_balanced_dataset('../data/raw/reviews.csv', target_size=TARGET_SIZE)
    if df.empty: return

    client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

    print(f"\n--- Старт генерации ({len(df)} отзывов) ---")

    output_file = "train_dataset.jsonl"

    with open(output_file, 'w', encoding='utf-8') as f:
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            text = row['text']

            # Генерация
            result = generate_label_safe(text, client)

            if result:
                # Формирование итоговой записи для обучения
                train_entry = {
                    "instruction": "Извлеки аспекты (delivery, price, quality, functionality, service) и тональность.",
                    "input": text,
                    "output": json.dumps(result, ensure_ascii=False)
                }

                # Запись в файл и в память
                f.write(json.dumps(train_entry, ensure_ascii=False) + "\n")
                f.flush()  # Принудительное сохранение на диск

            time.sleep(2)

    print(f"\n✅ Готово! Данные сохранены в {output_file}")

In [18]:
if __name__ == "__main__":
    main()

--- Подготовка данных ---
Всего доступно: Негатив=3057, Позитив=27133
Берем по 3000 отзывов каждого типа...
Итоговый датасет: 6000 строк.

--- Старт генерации (6000 отзывов) ---


 15%|█▍        | 874/6000 [1:14:29<5:32:50,  3.90s/it] 

Error: Connection error.


 64%|██████▎   | 3811/6000 [5:08:28<5:15:13,  8.64s/it] 

Error: 'NoneType' object is not subscriptable


100%|██████████| 6000/6000 [8:13:52<00:00,  4.94s/it]    


✅ Готово! Данные сохранены в train_dataset_strict_demo.jsonl


#### Валидация датасета

In [8]:
def validate_dataset(file_path, limit=50):

    errors = 0
    valid_count = 0

    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if limit and i >= limit:
                break

            try:
                # Проверка структуры строки (JSONL)
                row = json.loads(line)

                # Проверка ключей Alpaca
                if not all(k in row for k in ["instruction", "input", "output"]):
                    print(f"Line {i}: Отсутствуют ключи instruction/input/output")
                    errors += 1
                    continue

                # Проверка вложенного JSON
                output_str = row['output']
                if not isinstance(output_str, str):
                    print(f"Line {i}: Поле 'output' должно быть строкой (str), а не dict!")
                    errors += 1
                    continue

                # Попытка распарсить output (Inner JSON)
                try:
                    output_json = json.loads(output_str)
                except json.JSONDecodeError:
                    print(f"Line {i}: Поле 'output' содержит битый JSON.\nRaw: {output_str}")
                    errors += 1
                    continue

                # Проверка логики (Aspects existence)
                if "aspects" not in output_json or "sentiment" not in output_json:
                    print(f"Line {i}: В output нет обязательных полей sentiment/aspects.")
                    errors += 1
                    continue

                # Проверка значений (Pydantic-style check manual)
                valid_sentiments = ["positive", "negative", "neutral"]
                if output_json.get("sentiment") not in valid_sentiments:
                     print(f"Line {i}: Странный sentiment: {output_json.get('sentiment')}")

                valid_count += 1

            except Exception as e:
                print(f"Line {i}: Критическая ошибка парсинга: {e}")
                errors += 1

    print(f"Итог проверки первых {limit} строк:")
    print(f"Валидных: {valid_count}")
    print(f"Ошибок:   {errors}")

In [9]:
validate_dataset("train_dataset.jsonl", limit=TARGET_SIZE)

Итог проверки первых 6000 строк:
Валидных: 6000
Ошибок:   0
